<a href="https://colab.research.google.com/github/davetew/Modern-Aerospace-Propulsion/blob/main/Week%2007%20-%20Advanced%20Cycles/SimpleTurboFan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Turbo Fan
`SimpleTurboFan` is intended to facilitate estimates of the overall efficiency and massflow-specific thrust of turbofan engines given the required design and operating parameters.

The design parameters include--
1. $BypRatio$: the bypass to core mass flow ratio
2. $OPR$: the overall pressure ratio--compressor exit total to fan inlet total,
4. $FPR$: the fan pressure ratio--fan exit total to fan inlet total,
5. $η^{poly}_{fan}$: the fan polytropic efficiency,
6. $η^{poly}_{compressor}$: the compressor polytropic efficiency,
7. $η^{poly}_{turbine}$: the turbine polytropic efficiency,
8. $T^{metal}_{turbine}$: the turbine metal temperature,
9. $St_{turbine}$: the Stanton number for turbine cooling heat transfer,
10. $PR_{inlet}$: the inlet total pressure ratio,
11. $PR_{combustor}$: the combustor total pressure ratio,
12. $PR_{nozzle}$: the core/primary nozzle inflow total to exit static pressure ratio

The operating parameters include--
1. $M_{flight}$: the flight Mach number
2. $p_{ambient}$: the ambient static pressure
3. $T_{ambient}$: the ambient static temperature
4. $R_{ambient}$: the ambient gas-specific gas constant in $\frac{J}{kgK}$
5. $\gamma$: the ambient ratio of specfic heats

### Thrust Calculation

The net thrust of the propulsion system is given by

$$T = \dot{m}_{bypass} U^{exit}_{fan} + \dot{m}_{core}U^{exit}_{core} - \left( \dot{m}_{bypass} + \dot{m}_{core} \right) U_{\infty} $$


In [ ]:
!pip install -q cantera pint ambiance

In [ ]:
# Import the necessary libraries
import numpy as np
from scipy import constants
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import pint, cantera as ct, requests
from io import BytesIO
from scipy.optimize import minimize, brentq, NonlinearConstraint, fsolve, root
from ambiance import Atmosphere
from copy import deepcopy
from typing import Union, List, Tuple, Dict, Optional, Literal

# Save default unit registry in ureg & Quantity in Q_
ureg = pint.UnitRegistry()
Q_ = ureg.Quantity

# Import useful constants from scipy
from scipy import constants

# Add pint units to the constants
class u_constants:
    """Useful constants with pint units"""
    R = Q_(constants.R, 'J/(mol*K)')
    F = Q_(constants.physical_constants['Faraday constant'][0], 'C/mol')
    g = Q_(constants.g, 'm/s**2')

# Define a currency dimension with usd (US dollar) as the base unit
ureg.define('usd = [currency]')

# Define a passenger dimension with passengers as the base unit
ureg.define('passenger = [passenger]')

# Extract the magnitudes for each Quantity in Qlist
mag = lambda Qlist, units: [Q.to(units).magnitude for Q in Qlist]

# Ignore pint unit strip warnings
import warnings
warnings.filterwarnings("ignore", message="The unit of the quantity is stripped.")

def debug_io(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned {result}")
        return result
    return wrapper

In [ ]:
# Compressible flow relations

def Tt_T(Mach, γ=1.4):
  """Total to static temperature ratio"""
  return 1 + (γ - 1)/2 * Mach**2

def Tstatic(Tt, Mach, γ=1.4):
  """Static temperature"""
  return Tt / ( 1 + (γ-1)/2*Mach**2)

def Pt_p(Mach, γ=1.4):
  """Total to static pressure ratio"""
  return Tt_T(Mach, γ)**(γ/(γ-1))

def calcMach(Tt_t, γ=1.4):
  """Mach number"""
  return np.sqrt( 2/(γ-1)*(Tt_t - 1 ) )

# Turbomachinery exit conditions
def compExitT(InletT, PR, ηpoly, γ=1.4):
  """Calculate the compressor exit temperature given
    1. the inlet temperature,
    2. the pressure ratio,
    3. the polytropic efficiency, and
    4. the ratio of specific heats"""
  return InletT*PR**((γ-1)/γ/ηpoly)


In [ ]:
# Simple Turbo Fan Class Definition
class SimpleTurboFan:

  def __init__(self, InletMach=0.8, BypRatio=None,
               PR = {'inlet': None, 'fan': 1.2, 'overall': 30,
                     'burner': 0.98, 'cooling': 0.95, 'nozzle': 1.5},
               ηpoly = {'fan': 0.9, 'compressor': 0.85, 'turbine': 0.9},
               altitude=Q_(0, 'm'),
               turbine = {'inletT': Q_(1200, 'degC').to('K'), 'metalT': Q_(900, 'degC').to('K'), 'StantonNum': 0.07}):
    """Define an instance of SimpleTurbonFan given
    1. InletMach: the fan inlet/flight Mach number,
    2. BypRatio: the fan to core mass flow ratio,
    3. OPR: the overall pressure ratio--compressor exit total to fan inlet total,
    4. FPR: the fan pressure ratio--fan exit total to fan inlet total,
    5. ηpoly_fan: the fan polytropic efficiency,
    6. ηpoly_compressor: the compressor polytropic efficiency,
    7. ηpoly_turbine: the turbine polytropic efficiency,
    8. T_ambient_C: the ambient static temperature in deg C,
    9. p_ambient_kPa: the ambient static pressure in kPa,
    10. R_J_kgK: the ambient/inflow gas-specific gas constant in J/kg/K,
    11. γ: the ambient/inflow ratio of specific heats,
    13. Turbine_Metal_Temperature_C: ,
    14. StantonNumber:
    15. inletPR: the inlet total pressure ratio,
    16: burnerPR: the burner/combustor total pressure ratio,
    17: coolingPR: the total pressure ratio across the cooling system--between the compressor exit and the turbine blades,
    18: nozzlePR: the core exhaust nozzle total to static pressure ratio.
    """

    self.InletMach = InletMach
    self.PR = PR
    self.ηpoly = ηpoly

    self.altitude = altitude
    self.ambient = {'T': Atmosphere(altitude.m_as('m')).temperature[0] * ureg('K'),
                    'p': Atmosphere(altitude.m_as('m')).pressure[0] * ureg('Pa'),
                    'R': Q_(287.058, 'J/kg/K'),  # Specific gas constant for air
                    'γ': 1.4}
    self.ambient['γ1'] = self.ambient['γ'] / (self.ambient['γ']-1)
    self.ambient['cp'] =  self.ambient['R'] * self.ambient['γ1']

    self.turbine = turbine

    if self.turbine['metalT'] is None:
      self.turbine['metalT'] = turbine['inletT']

    self.T_combustor_exit = turbine['inletT']

    if BypRatio is None and PR['fan'] is None:
      raise ValueError('Either BypRatio or FPR must be specified.')
    elif BypRatio is None:
      # Calculate the bypass ratio that the core is capable of enabling
      self.BypRatio = self.calcBypRatio()
    elif PR['fan'] is None:
      # Calculate the fan pressure ratio that the core is capable of enabling
      self.BypRatio = BypRatio
      self.PR['fan'] = self.calcFanPressureRatio()
    else:
      raise ValueError('Specify either BypRatio or FPR.')

  def calcBypRatio(self):
    """Calculate the bypass ratio that results in no net fan-compressor-turbine shaft work"""

    def δwork(BypRatio):
      #print(f'BypRatio Guess = {BypRatio}')
      return (self.core_mass_specific_work - self.fan_core_mass_specific_work(BypRatio)).magnitude

    # Solve for the bypass ratio that results in no net shaft work
    bypratio, results = brentq(δwork, 1e-6, 100, full_output=True)

    # Return an answer if the solver converges
    return bypratio if results.converged else np.nan

  def calcFanPressureRatio(self):
    """Calculate the fan pressure ratio that results in no net fan-compressor-turbine shaft work"""

    def δwork(PR_fan):
      self.PR['fan'] = PR_fan
      return (self.core_mass_specific_work - self.fan_core_mass_specific_work(self.BypRatio)).magnitude

    #def subsonic_fan_exit_mach_constraint(PR_fan):
    #  """Constraint for the fan pressure ratio"""
    #  self.PR['fan'] = PR_fan
    #  return 1.0 - self.M_fan_exit.magnitude

    # Solve for the fan pressure ratio that results in no net shaft work
    PR_fan, results = brentq(δwork, 1.0+1e-6, 20.0, full_output=True)
    self.PR['fan'] = PR_fan

    #results = minimize(δwork, [2.0], method='trust-constr', bounds=[(1.0+1e-6, 20.0)],
    #                   constraints=[{'type': 'ineq', 'fun': subsonic_fan_exit_mach_constraint}], tol=1e-3)
    #PR_fan = results.x[0]

    # Return an answer if the solver converges
    return PR_fan if results.converged else np.nan

  def optimize(self):
    """Optimize the cycle for maximum efficiency subject to the component efficiency and material
    temperature limits provided"""
    print('Optimizing turbofan for maximum overall efficiency . . . ')

    def stf(x):
      """Return an instance of SimpleTurboFan updated with the parameters in x"""

      # Copy the input params into the appropriate dictionary for initiation of an instance of SimpleTurboFan
      # x = [Fan PR, Compressor PR, TIT]
      PR = deepcopy(self.PR); PR['fan'] = x[0]; PR['overall'] = x[1]
      turbine = deepcopy(self.turbine); turbine['inletT'] = Q_(x[2], 'K')
      #print(f"FPR = {x[0]}, OPR={x[1]}, TIT (K)={x[2]}")

      return SimpleTurboFan(InletMach=self.InletMach,
                            BypRatio=None, # Bypass ratio will be calculated within the new instance
                            PR=PR,
                            ηpoly = self.ηpoly,
                            altitude=self.altitude,
                            turbine = turbine)

    def inefficiency(x):
      """Calculate and return the 'inefficiency' or 1 - efficiency"""
      return 1 - stf(x).overall_efficiency.magnitude

    results = minimize(inefficiency,
                        [self.PR['fan'], self.PR['overall'], self.turbine['inletT'].magnitude],
                        method='trust-constr', tol=1e-3,
                        bounds=[(1.15, 2), (1.0+1e-6, 100), (200, 1800)],
                        constraints=[NonlinearConstraint(lambda x: stf(x).M_fan_exit.magnitude, 0.0, max(1, self.InletMach + 0.5)),
                                     NonlinearConstraint(lambda x: (stf(x).fan_thrust / stf(x).thrust).m_as(""), 0.0, 1.0)],
                        options={'disp': True})

    if results.status == 1 or results.status == 2:
      self.PR['fan'] = results.x[0]
      self.PR['overall'] = results.x[1]
      self.T_combustor_exit = self.turbine['inletT'] = Q_(results.x[2], 'K')
      self.BypRatio = self.calcBypRatio()
      #print(f'results.x = {results.x}, Bypass Ratio = {self.BypRatio}')
      return self
    else:
      return None

  def sonicVel(self, T, γ=None, R=None):
    """Speed of sound in m/s"""

    # Ambient conditions are default
    γ = self.ambient['γ'] if γ is None else γ
    R = self.ambient['R'] if R is None else R

    return np.sqrt(γ*R*T).to('m/s')

  @property
  def U_flight(self):
    """Flight velocity in m/s"""
    return self.InletMach*self.sonicVel(self.ambient["T"]).to('m/s')

  @property
  def T_inlet(self):
    """Inlet total temperature in K"""
    return ( self.ambient['T']*Tt_T(self.InletMach, self.ambient['γ'])).to('K')

  @property
  def pi_inlet(self):
    """Inlet total pressure ratio"""
    if self.PR['inlet'] is not None:
      return self.PR['inlet']
    else:
      return 1 if self.InletMach < 1.0 else 1 - 0.075*(self.InletMach - 1)**1.35

  @property
  def P_inlet(self):
    """Inlet total pressure in Pa"""
    return ( self.ambient['p']*Pt_p(self.InletMach, self.ambient['γ']) * self.pi_inlet).to('Pa')

  @property
  def T_fan_exit(self):
    """Fan exit total temperature in K"""
    return compExitT(self.T_inlet, self.PR['fan'], self.ηpoly["fan"], self.ambient["γ"]).to('K')

  @property
  def P_fan_exit(self):
    """Fan exit total pressure in Pa"""
    return ( self.P_inlet*self.PR['fan'] ).to('Pa')

  @property
  def T_compressor_exit(self):
    """Compressor exit total temperature in K"""
    return compExitT(self.T_fan_exit, self.PR['overall']/self.PR['fan'], self.ηpoly["compressor"], self.ambient["γ"]).to('K')

  @property
  def P_compressor_exit(self):
    """Compressor exit total pressure in Pa"""
    return ( self.P_inlet*self.PR['overall'] ).to('Pa')

  @property
  def P_combustor_exit(self):
    """Combustor exit total pressure in Pa"""
    return (self.P_compressor_exit*self.PR['burner']).to('Pa')

  @property
  def βcooling(self):
    """Turbine cooling / inlet mass flow ratio"""
    return ( self.turbine['StantonNum']*(self.T_combustor_exit-self.turbine['metalT']) /
            (self.turbine['metalT'] - self.T_compressor_exit) )

  @property
  def P_turbine_exit(self):
    """Turbine exit total pressure in Pa"""
    return self.PR['nozzle']*self.ambient['p'].to('Pa')

  @property
  def turbinePR(self):
    """Turbine inlet to exhaust total pressure ratio"""
    return ( self.P_combustor_exit*(1-self.βcooling) + self.P_compressor_exit*self.βcooling ) / self.P_turbine_exit

  @property
  def T_turbine_inlet(self):
    """Temperature after mixing of combustor exit and cooling flows in K"""
    return ( self.T_compressor_exit*self.βcooling + self.T_combustor_exit*(1-self.βcooling) ).to('K')

  @property
  def T_turbine_exit(self):
    """Turbine exit temperature in K"""
    return ( self.T_turbine_inlet*self.turbinePR**(-(self.ambient["γ"]-1)*self.ηpoly["turbine"]/self.ambient["γ"]) ).to('K')

  @property
  def mass_specific_heat_addition(self):
    """Core mass specific heat addition in J/kg"""
    return ( (self.T_combustor_exit - self.T_compressor_exit)*self.ambient["cp"]*(1-self.βcooling) ).to('J/kg')

  def fan_core_mass_specific_work(self, BypRatio=None):
    BypRatio = self.BypRatio if BypRatio is None else BypRatio
    """Calculate the fan core-mass-specific input work in J/kg of core flow"""
    return ( self.ambient["cp"]*(1+BypRatio)*(self.T_fan_exit - self.T_inlet) ).to('J/kg')

  @property
  def compressor_specific_work(self):
    """Calculate the compessor mass-specific input work in J/kg of core flow"""
    return ( self.ambient["cp"]*(self.T_compressor_exit - self.T_fan_exit) ).to('J/kg')

  @property
  def turbine_specific_work(self):
    """Calculate the turbine mass-specific output work in J/kg of core flow"""
    return ( self.ambient["cp"]*(self.T_turbine_inlet - self.T_turbine_exit) ).to('J/kg')

  @property
  def core_mass_specific_work(self):
    """Net mass specific work in J/kg of core flow"""
    return ( self.turbine_specific_work - self.compressor_specific_work ).to('J/kg')

  @property
  def M_fan_exit(self):
    """Fan exit Mach number"""
    return calcMach( (self.P_fan_exit/self.ambient["p"])**(1/self.ambient["γ1"]), self.ambient["γ"] )

  @property
  def U_fan_exit(self):
    """Fan exit velocity"""
    return self.M_fan_exit*self.sonicVel(Tstatic(self.T_fan_exit, self.M_fan_exit, self.ambient["γ"]))

  @property
  def M_core_exit(self):
    """Core nozzle exit Mach number"""
    return calcMach( (self.P_turbine_exit/self.ambient["p"])**(1/self.ambient["γ1"]), self.ambient["γ"] )

  @property
  def U_core_exit(self):
    """Core exit velocity in m/s"""
    return ( self.M_core_exit*self.sonicVel(Tstatic(self.T_turbine_exit, self.M_core_exit, self.ambient["γ"])) ).to('m/s')

  @property
  def core_thrust(self):
    """Core thrust non-dimensionalized by the "core inflow" momentum flux"""
    return self.U_core_exit / self.U_flight - 1

  @property
  def fan_thrust(self):
    """Core thrust non-dimensionalized by the "core inflow" momentum flux"""
    return self.BypRatio*( self.U_fan_exit/self.U_flight - 1 )

  @property
  def thrust(self):
    """Thrust non-dimensionalized by the "core inflow" momentum flux"""
    return self.fan_thrust + self.core_thrust

  @property
  def ΔKE(self):
    """Change in kinetic energy in J/kg of core flow"""
    return 0.5*( self.BypRatio*(self.U_fan_exit**2 - self.U_flight**2) + self.U_core_exit**2 - self.U_flight**2).to('J/kg')

  @property
  def thermal_efficiency(self):
    """Core thermal efficiency"""
    return (self.ΔKE / self.mass_specific_heat_addition).to('dimensionless')

  @property
  def overall_efficiency(self):
    """Overall propulsion system efficiency"""
    return ( self.thrust*self.U_flight**2/self.mass_specific_heat_addition ).to('dimensionless')

  @property
  def propulsive_efficiency(self):
    """Propulsive efficiency"""
    return self.overall_efficiency / self.thermal_efficiency

  @property
  def cycleTemperatures(self):
    return [self.T_inlet, self.T_fan_exit, self.T_compressor_exit,
                    self.T_combustor_exit, self.T_turbine_inlet, self.T_turbine_exit]

  @property
  def cyclePressures(self):
    """Cycle total pressures"""
    return [self.P_inlet, self.P_fan_exit, self.P_compressor_exit,
                    self.P_combustor_exit, self.P_combustor_exit, self.P_turbine_exit]

  @property
  def cycleEnthalpies(self):
    return [self.enthalpy(T) for T in self.cycleTemperatures]

  @property
  def cycleEntropies(self):
    return [self.entropy(T, p) for T, p in zip(self.cycleTemperatures, self.cyclePressures)]

  @property
  def cycleMach(self):
    noMach = Q_(np.nan, 'dimensionless')
    return [self.InletMach*ureg('dimensionless'), self.M_fan_exit, noMach, noMach, noMach, self.M_core_exit]

  def enthalpy(self, T):
    """Calcuate and return the mass-specific enthalpy in J/kg"""
    return ( self.ambient["cp"]*(T - self.ambient["T"]) ).to('J/kg')

  def entropy(self, T, p):
    """Calcuate and return the mass-specific entropy in J/kg/K"""
    return ( self.ambient["cp"]*np.log(T/self.ambient["T"]) -
            self.ambient["R"]*np.log(p/self.ambient["p"]) ).to('J/kg/K')

  @property
  def statePoints(self):
    """Return a dataframe with the thermodynamic states at each station"""
    stations = ['Inlet', 'Fan Exit', 'Compressor Exit', 'Combustor Exit', 'Mixed Turbine Inlet', 'Turbine Exit']
    stateData = {'Temperature (C)': mag(self.cycleTemperatures, 'degC'),
            'Pressure (kPa)': mag(self.cyclePressures, 'kPa'),
            'Enthalpy (kJ/kg)': mag(self.cycleEnthalpies, 'kJ/kg'),
            'Entropy (kJ/kg/K)': mag(self.cycleEntropies, 'kJ/kg/K'),
            'Mach Number': mag(self.cycleMach, 'dimensionless')}
    return pd.DataFrame(data=stateData,index=stations)

  @property
  def cycleSummary(self):
    """Return a dataframe with a summary of the cycle design specifications & performance"""
    cycleData = {'Ambient Temperature (C)': self.ambient['T'].to('degC').magnitude,
                 'Ambient Pressure (kPa)': self.ambient['p'].to('kPa').magnitude,
                 'Inlet Mach Number': self.InletMach,
                 'Fan Pressure Ratio': self.PR['fan'],
                 'Bypass Ratio': self.BypRatio,
                 'Overall Pressure Ratio': self.PR['overall'],
                 'Turbine Inlet Temperature (C)': self.turbine['inletT'].to('degC').magnitude,
                 'Turbine Metal Temperature (C)': self.turbine['metalT'].to('degC').magnitude,
                 'Stanton Number': self.turbine['StantonNum'],
                 'Turbine Cooling / Inlet Mass Flow Ratio': self.βcooling.magnitude,
                 'Thrust / Fan Inlet Momentum Flux': self.thrust.magnitude / self.BypRatio,
                 'Fan / Total Thrust': ( self.fan_thrust / self.thrust ).magnitude,
                 'Overall Efficiency': self.overall_efficiency.magnitude,
                 'Thermal Efficiency': self.thermal_efficiency.magnitude,
                 'Propulsive Efficiency': self.propulsive_efficiency.magnitude}

    return pd.Series(data=cycleData)


  def cycleDiagrams(self):

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,6))

    # Temperature-Entropy Diagram
    ax1.plot(mag(self.cycleEntropies, 'J/kg/K'), mag(self.cycleTemperatures, 'degC'), marker='d')
    ax1.set_xlabel('Entropy (J/kg/K)')
    ax1.set_ylabel('Temperature (C)')
    ax1.grid()
    ax1.set_title(f'TS: Efficiency= {self.overall_efficiency.magnitude*100:0.0f}%')

    # Pressure-Enthalpy
    ax2.plot(mag(self.cycleEnthalpies,'kJ/kg'), mag(self.cyclePressures,'kPa'), marker='p')
    ax2.set_xlabel('Enthalpy (kJ/kg)')
    ax2.set_ylabel('Pressure (kPa)')
    ax2.grid()
    ax2.set_title(f'PH: Specific Work={self.core_mass_specific_work.to("kJ/kg").magnitude:.0f} kJ/kg')


In [ ]:
olympus = SimpleTurboFan(InletMach=0.33, BypRatio=None,
                         PR={ 'inlet': None,'fan': 1.2,'overall': 30,'burner': 0.98,'cooling': 0.95,'nozzle': 1.5 },
                         altitude=Q_(0, 'm'),
                         turbine = {'inletT': Q_(1500, 'degC').to('K'), 'metalT': Q_(1000, 'degC').to('K'), 'StantonNum': 0.07})
opt = olympus.optimize()
display(opt.cycleSummary)
display(opt.statePoints)


Optimizing turbofan for maximum overall efficiency . . . 
`gtol` termination condition is satisfied.
Number of iterations: 5, function evaluations: 16, CG iterations: 4, optimality: 9.08e-04, constraint violation: 0.00e+00, execution time: 0.54 s.


,0
Ambient Temperature (C),15.000000
Ambient Pressure (kPa),101.325000
Inlet Mach Number,0.330000
Fan Pressure Ratio,1.261509
Bypass Ratio,13.015080
Overall Pressure Ratio,29.998771
Turbine Inlet Temperature (C),1500.007536
Turbine Metal Temperature (C),1000.000000
Stanton Number,0.070000
Turbine Cooling / Inlet Mass Flow Ratio,0.098996


,Temperature (C),Pressure (kPa),Enthalpy (kJ/kg),Entropy (kJ/kg/K),Mach Number
Inlet,21.275907,109.261587,6.305423,3.552714e-18,0.330000
Fan Exit,43.810157,137.834443,28.945651,7.409554e-03,0.677867
Compressor Exit,646.446582,3277.713318,634.416275,1.679348e-01,NaN
Combustor Exit,1500.007536,3212.159052,1491.991526,8.334042e-01,NaN
Mixed Turbine Inlet,1415.508045,3212.159052,1407.094635,7.843468e-01,NaN
Turbine Exit,497.052312,151.987500,484.319404,8.714039e-01,0.783659
